#Linear regression using PyTorch built-ins

In [1]:
import numpy as np
import torch

In [4]:
# Targets (apples, oranges)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [5]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [6]:
from torch.utils.data import TensorDataset

In [7]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [8]:
from torch.utils.data import DataLoader

In [9]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [10]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [103., 119.]])


#nn.Linear

In [11]:
import torch.nn as nn

In [12]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2493,  0.0055, -0.2316],
        [ 0.1628, -0.1556, -0.5479]], requires_grad=True)
Parameter containing:
tensor([ 0.1661, -0.0825], requires_grad=True)


In [14]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2493,  0.0055, -0.2316],
         [ 0.1628, -0.1556, -0.5479]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1661, -0.0825], requires_grad=True)]

In [15]:
preds = model(inputs)
preds

tensor([[  8.7794, -22.1806],
        [  8.5197, -34.0233],
        [  9.1671, -38.5452],
        [ 17.2672, -10.4359],
        [  1.6886, -42.1384]], grad_fn=<AddmmBackward0>)

#loss functions

In [18]:
import torch.nn.functional as F
loss_fn = F.mse_loss

In [19]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(11420.4902, grad_fn=<MseLossBackward0>)


#Optimizer

In [20]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)


#Train for multiple epochs

In [21]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [22]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1058.9485
Epoch [20/100], Loss: 678.9884
Epoch [30/100], Loss: 600.3947
Epoch [40/100], Loss: 535.9902
Epoch [50/100], Loss: 479.2323
Epoch [60/100], Loss: 429.1173
Epoch [70/100], Loss: 384.8532
Epoch [80/100], Loss: 345.7440
Epoch [90/100], Loss: 311.1770
Epoch [100/100], Loss: 280.6121


In [23]:
prediction = model(inputs)
prediction

tensor([[ 62.3297,  75.6192],
        [ 81.5875,  97.4226],
        [111.7995, 131.8072],
        [ 50.5220,  67.2659],
        [ 83.6956,  95.8197]], grad_fn=<AddmmBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [25]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[60.0324, 73.1852]], grad_fn=<AddmmBackward0>)